In [1]:
import pandas as pd
import numpy as np
import requests
import googlemaps
API_KEY = 'api key'
gmaps_dev = googlemaps.Client(key=API_KEY)

ValueError: Invalid API key provided.

In [2]:
df = pd.read_csv('crime_final_clean10_17.csv')

In [3]:
selected_beats = ['3B10','1A10','10H30','10H50','10H60','10H80','15E40' ]

In [4]:
df.head()

,Date,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year
0,1914-09-08,24C60,12700-12799,LAKE HOUSTON,Burglary,Restaurant or Cafeteria,1.0,7.0,8,Tuesday,Sep,1914
1,1914-11-02,18F60,8800-8899,BELLAIRE,Burglary,Miscellaneous Business (Non-Specific),1.0,3.0,2,Monday,Nov,1914
2,1914-12-03,12D20,12800-12899,GULF,Auto Theft,No label,1.0,19.0,3,Thursday,Dec,1914
3,1915-01-05,3B10,3200-3299,MANGUM RD 180,Theft,Other Parking Lot,1.0,NaN,5,Tuesday,Jan,1915
4,1915-01-14,5F10,7000-7099,WESTVIEW,Auto Theft,Apartment Parking Lot,1.0,NaN,14,Thursday,Jan,1915


In [5]:
stadium_beats = df[df['Beat'].isin(selected_beats)]

In [6]:
stadium_beats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77808 entries, 3 to 1006323
Data columns (total 12 columns):
Date           77808 non-null object
Beat           77808 non-null object
BlockRange     77685 non-null object
StreetName     77808 non-null object
OffenseType    77808 non-null object
Premise        77808 non-null object
NumOffenses    77808 non-null float64
Hour           4978 non-null float64
day            77808 non-null int64
weekday        77808 non-null object
month          77808 non-null object
year           77808 non-null int64
dtypes: float64(2), int64(2), object(8)
memory usage: 7.7+ MB


## remove null values on blockrange

In [7]:
stadium_beats = stadium_beats[stadium_beats.BlockRange.notnull()]

In [8]:
stadium_beats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77685 entries, 3 to 1006323
Data columns (total 12 columns):
Date           77685 non-null object
Beat           77685 non-null object
BlockRange     77685 non-null object
StreetName     77685 non-null object
OffenseType    77685 non-null object
Premise        77685 non-null object
NumOffenses    77685 non-null float64
Hour           4875 non-null float64
day            77685 non-null int64
weekday        77685 non-null object
month          77685 non-null object
year           77685 non-null int64
dtypes: float64(2), int64(2), object(8)
memory usage: 7.7+ MB


### Create a temporary address column

In [9]:
stadium_beats['temp_add'] = stadium_beats[['BlockRange', 'StreetName']].apply(lambda x: ' '.join(x), axis=1)

In [10]:
stadium_beats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77685 entries, 3 to 1006323
Data columns (total 13 columns):
Date           77685 non-null object
Beat           77685 non-null object
BlockRange     77685 non-null object
StreetName     77685 non-null object
OffenseType    77685 non-null object
Premise        77685 non-null object
NumOffenses    77685 non-null float64
Hour           4875 non-null float64
day            77685 non-null int64
weekday        77685 non-null object
month          77685 non-null object
year           77685 non-null int64
temp_add       77685 non-null object
dtypes: float64(2), int64(2), object(9)
memory usage: 8.3+ MB


In [11]:
stadium_beats['result'] = None

In [12]:
stadium_beats.head()

,Date,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,temp_add,result
3,1915-01-05,3B10,3200-3299,MANGUM RD 180,Theft,Other Parking Lot,1.0,NaN,5,Tuesday,Jan,1915,3200-3299 MANGUM RD 180,None
40,1966-01-01,10H50,3300-3399,ALABAMA,Rape,APARTMENT,1.0,NaN,1,Saturday,Jan,1966,3300-3399 ALABAMA,None
47,1971-02-03,1A10,1200-1299,TRAVIS,Theft,Residence or House,1.0,6.0,3,Wednesday,Feb,1971,1200-1299 TRAVIS,None
65,1977-10-14,10H50,3200-3299,TRUXILLO,Theft,APARTMENT PARKING LOT,1.0,NaN,14,Friday,Oct,1977,3200-3299 TRUXILLO,None
76,1980-01-17,3B10,4900-4999,DACOMA,Theft,Apartment Parking Lot,1.0,NaN,17,Thursday,Jan,1980,4900-4999 DACOMA,None


In [13]:
stadium_beats.Beat.value_counts()

3B10     18251
15E40    16434
1A10     15329
10H50     8042
10H60     7717
10H80     7503
10H30     4409
Name: Beat, dtype: int64

## Divide beats

In [14]:
beat_3B10 = stadium_beats[stadium_beats.Beat == '3B10']
beat_15E40 = stadium_beats[stadium_beats.Beat == '15E40']
beat_1A10 = stadium_beats[stadium_beats.Beat == '1A10']
beat_10H50 = stadium_beats[stadium_beats.Beat == '10H50']
beat_10H60 = stadium_beats[stadium_beats.Beat == '10H60']
beat_10H80 = stadium_beats[stadium_beats.Beat == '10H80']
beat_10H30 = stadium_beats[stadium_beats.Beat == '10H30']

## get lat long and full address form temp_add

In [ ]:
def get_geocode(loc,API_KEY):
    '''input address and api key
    return tuple with lat and long'''
    address = '{}, Houston, TX'.format(loc)
    pa = {'address': address, 'key':API_KEY}
    URL = 'https://maps.googleapis.com/maps/api/geocode/json'
    response = requests.get(URL,params=pa)
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError as e:
        return "Error: " + str(e) # not 200
    js_obj = response.json()
    if js_obj['status'] == 'OK':
        gps = tuple(js_obj['results'][0]['geometry']['location'].values())
        full_add = js_obj['results'][0]['formatted_address']
        return gps[0],gps[1],full_add
    elif js_obj['status'] =='OVER_QUERY_LIMIT':
        return "OVER_QUERY_LIMIT"
    else:
        print(js_obj['status'])
        return js_obj['status']

In [15]:
def get_geocode(address,API_KEY):
    loc = '{}, Houston, TX'.format(address)
    gmaps = googlemaps.Client(key=API_KEY)
    r = gmaps.geocode(loc)
    lat_lng = tuple(r[0]['geometry']['location'].values())
    full_add = r[0]['formatted_address']
    return lat_lng[0],lat_lng[1], full_add

In [20]:
beat_10H30.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 368 to 1006250
Data columns (total 14 columns):
Date           4409 non-null object
Beat           4409 non-null object
BlockRange     4409 non-null object
StreetName     4409 non-null object
OffenseType    4409 non-null object
Premise        4409 non-null object
NumOffenses    4409 non-null float64
Hour           184 non-null float64
day            4409 non-null int64
weekday        4409 non-null object
month          4409 non-null object
year           4409 non-null int64
temp_add       4409 non-null object
result         0 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 676.7+ KB


In [18]:
beat_10H30['result'] = beat_10H30['temp_add'].apply(get_geocode,args=(API_KEY,))

Timeout: 

In [19]:
beat_10H30

,Date,Beat,BlockRange,StreetName,OffenseType,Premise,NumOffenses,Hour,day,weekday,month,year,temp_add,result
368,2001-12-30,10H30,2000-2099,TEXAS,Aggravated Assault,APARTMENT,2.0,NaN,30,Sunday,Dec,2001,2000-2099 TEXAS,None
443,2004-01-09,10H30,2700-2799,CLAY,Theft,Residence or House,1.0,NaN,9,Friday,Jan,2004,2700-2799 CLAY,None
505,2005-02-12,10H30,800-899,ST EMANUEL,Theft,BAR/NIGHT CLUB,1.0,NaN,12,Saturday,Feb,2005,800-899 ST EMANUEL,None
566,2005-11-23,10H30,800-899,ST EMANUEL,Theft,BAR/NIGHT CLUB PARKING LOT,1.0,NaN,23,Wednesday,Nov,2005,800-899 ST EMANUEL,None
576,2005-12-12,10H30,800-899,ST EMANUEL,Theft,BAR/NIGHT CLUB,1.0,NaN,12,Monday,Dec,2005,800-899 ST EMANUEL,None
824,2008-03-22,10H30,400-499,DOWLING,Rape,No label,1.0,NaN,22,Saturday,Mar,2008,400-499 DOWLING,None
1080,2009-01-06,10H30,900-999,MCKINNEY PARK,Theft,DRIVEWAY,1.0,NaN,6,Tuesday,Jan,2009,900-999 MCKINNEY PARK,None
1321,2009-06-01,10H30,2700-2799,DALLAS,Theft,DRIVEWAY,1.0,NaN,1,Monday,Jun,2009,2700-2799 DALLAS,None
1407,2009-08-01,10H30,1700-1799,ENNIS,Theft,OTHER/UNKNOWN,1.0,NaN,1,Saturday,Aug,2009,1700-1799 ENNIS,None
1762,2009-11-26,10H30,2100-2199,POLK,Burglary,VACANT BUILDING (COMMERCIAL),1.0,NaN,26,Thursday,Nov,2009,2100-2199 POLK,None


## test on 10 rows

In [ ]:
beat_10H30['result'] = beat_10H30.iloc[:100]['temp_add'].apply(get_geocode,args=(API_KEY,))

In [ ]:
beat_10H30['result'] = beat_10H30.iloc[100:110]['temp_add'].apply(get_geocode,args=(API_KEY,))

In [ ]:
beat_10H30.info()

In [ ]:
beat_10H30.iloc[100:110]

In [ ]:
beat_10H30['result'] = beat_10H30.iloc[112:120]['temp_add'].apply(get_geocode,args=(API_KEY,))

In [21]:
beat_10H30.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 368 to 1006250
Data columns (total 14 columns):
Date           4409 non-null object
Beat           4409 non-null object
BlockRange     4409 non-null object
StreetName     4409 non-null object
OffenseType    4409 non-null object
Premise        4409 non-null object
NumOffenses    4409 non-null float64
Hour           184 non-null float64
day            4409 non-null int64
weekday        4409 non-null object
month          4409 non-null object
year           4409 non-null int64
temp_add       4409 non-null object
result         0 non-null object
dtypes: float64(2), int64(2), object(10)
memory usage: 676.7+ KB


In [26]:
beat_10H30.iloc[112:120]['temp_add'].apply(get_geocode,args=(API_KEY,))

Timeout: 

In [24]:
res

NameError: name 'res' is not defined